**When AI Lies With Confidence**
Hallucination Hunters: Shubhi Handa, Ron Levy, Mia Young, Evelyn Salas

Our Schema Categories: Confident Misinformation, Confident Accuracy, Neutral Misinformation, Neutral Accuracy, Unconfident Misinformation, Unconfident Accuracy

**Problem Statement**: In a world where we increasingly turn to AI for answers, a new problem has emerged: when these systems speak so convincingly, it becomes almost impossible to tell fact from fiction. LLMs often generate responses to queries with varying degrees of confidence. However, confidently delivered misinformation may be particularly harmful, especially in health contexts. This project aims to analyze when and how LLMs produce confident misinformation, particularly in response to women's reproductive health questions.

**Our Topic Lens**: We focus on women's reproductive health, including pregnancy, contraception, menstruation, hormonal conditions, postpartum care, etc.

**Our Questions**
How do LLMs express persuasiveness or certainty in responses that are factually inaccurate?

Are there linguistic markers of confidence that reliably correlate with accuracy or misinformation?

How can we classify or measure confidence based off of Hedging, Assertive, or Emotion/Persuasive words?

**Dataset**: We use the dataset LLM-Healthcare/trad_ai_medical_chatbot_15000_35400, loaded from HuggingFace

Shubhi: Found the dataset, cleaned and filtered dataset, identified 12 LIWC Categories, filtered for women's reproductive health with keywords, created scoring system to normalize scores and label confidence, manually analyzed 25 queries for accuracy/misinformation
Evelyn: Coded 6 categories for LIWC to use for analyzing confidence levels, met with Philosophy Professor to gather more information on LLMs and Confidence, manually analyzed 25 queries for accuracy/misinformation
Mia: Created Wix website to display our current work, manually analyzed 25 queries for accuracy/misinformation
Ron: Created Wix website to display our current work, manually analyzed 25 queries for accuracy/misinformation

In [ ]:
import csv
import pandas as pd
LLM_Health = pd.read_csv("NIH Questions - All-Responses (1).csv")


In [ ]:
# Preview to confirm
LLM_Health.head()


**Data Extracting**: We extracted the dataset to focus only on the response in between either " quotes or ' quotes.

In [ ]:
import re

# Extract text between the first pair of quotes
def extract_between_quotes(text):
    if isinstance(text, str):
        match = re.search(r'''(["'])(.*?)\1''', text, re.DOTALL)
        if match:
            return match.group(2).strip()
    return text

# Create a new DataFrame with the cleaned 'Answer' column
filteredLLM = LLM_Health.copy()
filteredLLM["Answer"] = filteredLLM["Answer"].apply(extract_between_quotes)

filteredLLM.head()

LIWC Dictionary columns to investigate for confidence levels: Certain, CogProc, Achieve, Power, Reward, Risk, Negate, Function, Auxverb, Affect, Posemo, Drives, Tentat

**LIWC Dictionary**: We imported the LIWC dictionary and converted it to json. Then cleaned the data by dropping the unnamed columns.

In [ ]:
import json

In [ ]:
import pandas as pd

# Load the CSV
df = pd.read_csv("LIWKDictionary.csv")

# Convert to list of dictionaries (or use df.to_dict('records'))
data_as_json = df.to_json(orient="records", indent=2)

# Create a dictionary for each category
liwc_dict = {}

# Save to JSON file
with open("LIWKDictionary.json", "w") as f:
    json.dump(liwc_dict, f)

print("Saved LIWC dictionary as JSON.")

# Drop columns with names that contain 'Unnamed' (likely irrelevant)
df_cleaned = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # Remove columns with 'Unnamed'
df_cleaned = df_cleaned.dropna(axis=0, subset=['2'])  # Remove rows that don't contain useful data

# Clean up column names by stripping leading/trailing spaces
df_cleaned.columns = df_cleaned.columns.str.strip()

# Ensure only useful rows and columns remain
df_cleaned = df_cleaned.dropna(axis=1, how='all')  # Remove columns with all NaN values

# Check the cleaned column names and a sample of the data
print(df_cleaned.columns)  # To see the final column names
print(df_cleaned.head())   # To verify the top rows of the cleaned DataFrame

**Creating the Dictionary**: We turned LIWC into a Dictionary by extracting the categories we wanted to focus on.

In [ ]:
# categories are in the first row, and their words are in the next rows.
categories = ["55\nCertain", "50\nCogProc", "82\nAchieve", "83\nPower", "84\nReward", "85\nRisk", "15\nNegate", "1 Function", "12\nAuxverb", "30\nAffect", "31\nPosemo", "80\nDrives", "54\nTentat"]

for category in categories:
    # Extract words for each category
    words = df_cleaned[category].dropna().tolist()  # Remove NaN values and convert to list
    liwc_dict[category] = words

print(liwc_dict)

Added more lists like booster or hedging words from additional research

In [ ]:
# Word lists
booster_words = [
   "absolute", "absolutely", "accura", "all", "altogether", "always", "apparent",
    "assur", "blatant", "certain", "clear", "clearly", "commit", "commitment", "commits", "committed", "committing",
    "complete", "completed", "completely", "completes", "confidence", "confident", "confidently",
    "correct", "defined", "definite", "definitely", "definitive", "directly", "distinct", "entire",
    "especially", "essential", "ever", "every", "everybod", "everyday", "everyone", "everything", "everytime", "everywhere",
    "evident", "exact", "explicit", "extremely", "fact", "facts", "factual", "forever", "frankly",
    "fundamental", "fundamentalis", "fundamentally", "fundamentals", "guarant", "implicit", "indeed", "inevitab",
    "infallib", "invariab", "irrefu", "must", "namely", "necessari", "necessary", "never",
    "nothing", "nowhere", "obvious", "obviously", "particularly", "perfect", "perfected", "perfecting", "perfection",
    "perfectly", "perfects", "positive", "positively", "positives", "positivi", "precis", "promise", "proof",
    "prove", "proving", "pure", "purely", "pureness", "purest", "purity", "specific", "specifically", "specifics",
    "sure", "total", "totally", "true", "truest", "truly", "truth", "unambigu", "undeniab", "undoubt",
    "unquestion", "visibly", "wholly"
]

hedging_words = [
    "might", "could", "may", "can", "would",
    "possibly", "perhaps", "apparently", "seemingly", "likely",
    "conceivably", "presumably", "reportedly",
    "suggest", "indicate", "appear", "seem", "imply", "speculate", "propose", "estimate",
    "possible", "probable", "potential", "uncertain", "likely"
]

superlatives = [
    "best", "greatest", "most important", "ultimate", "most significant", "highest", "largest", "leading"
]

**Buckets**: We then organized the LIWC and custom word lists into two Buckets: Hedging, and Assertive

In [ ]:
# Load the existing LIWC dictionary from the JSON file
with open("LIWKDictionary.json") as f:
    liwc_dict = json.load(f)

# Combine LIWC-based category word lists using the correct column names
hedging = (
    liwc_dict.get("54\nTentat", []) +   # Adjusted to match the new category names
    liwc_dict.get("12\nAuxverb", []) +
    liwc_dict.get("15\nNegate", [])
)

assertive = (
    liwc_dict.get("55\nCertain", []) +
    liwc_dict.get("82\nAchieve", []) +
    liwc_dict.get("83\nPower", []) +
    liwc_dict.get("84\nReward", []) +
    liwc_dict.get("85\nRisk", []) +
    liwc_dict.get("50\nCogProc", [])
)

pathos = (
    liwc_dict.get("30\nAffect", []) +
    liwc_dict.get("31\nPosemo", []) +
    liwc_dict.get("80\nDrives", [])
)

# Add custom word lists if necessary
hedging += hedging_words
assertive += booster_words
pathos += superlatives

# Now you can work with these combined lists, e.g., print them or process further
print("Hedging words:", hedging)
print("Assertive words:", assertive)
print("Pathos words:", pathos)
hedging_set = set(word.lower() for word in hedging)
assertive_set = set(word.lower() for word in assertive)
pathos_set = set(word.lower() for word in pathos)

custom_hedging_set = set(word.lower() for word in hedging_words)
custom_assertive_set = set(word.lower() for word in booster_words)
custom_pathos_set = set(word.lower() for word in superlatives)

# Step 3: Remove overlaps between LIWC and custom sets
custom_hedging_set -= hedging_set
custom_assertive_set -= assertive_set
custom_pathos_set -= pathos_set

# Step 4: Merge into final sets (no duplicates now)
final_hedging = hedging_set.union(custom_hedging_set)
final_assertive = assertive_set.union(custom_assertive_set)
final_pathos = pathos_set.union(custom_pathos_set)

Did some more cleaning to ensure no overlap

In [ ]:
hedging_set = set(word.lower() for word in hedging)
assertive_set = set(word.lower() for word in assertive)
pathos_set = set(word.lower() for word in pathos)

custom_hedging_set = set(word.lower() for word in hedging_words)
custom_assertive_set = set(word.lower() for word in booster_words)
custom_pathos_set = set(word.lower() for word in superlatives)

# Step 3: Remove overlaps between LIWC and custom sets
custom_hedging_set -= hedging_set
custom_assertive_set -= assertive_set
custom_pathos_set -= pathos_set

# Step 4: Merge into final sets (no duplicates now)
final_hedging = hedging_set.union(custom_hedging_set)
final_assertive = assertive_set.union(custom_assertive_set)
final_pathos = pathos_set.union(custom_pathos_set)


**spaCy**: used spaCy to get a simple measurement of the words across the responses.

In [ ]:
import spacy
from tqdm import tqdm

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")
tqdm.pandas()

# Lowercase LIWC word sets
final_hedging = set(word.lower() for word in final_hedging)
final_assertive = set(word.lower() for word in final_assertive)
final_pathos = set(word.lower() for word in final_pathos)

# Authority terms for ethos detection
authority_list = [
    "CDC", "WHO", "mayo clinic", "johns hopkins", "obstetrician", "gynecologist",
    "dr", "doctor", "physician", "clinician", "ACOG", "health department", "NIH"
]

# -- Helper functions using spaCy Docs --

def count_liwc_words(doc):
    hedging = sum(1 for token in doc if token.text.lower() in final_hedging)
    assertive = sum(1 for token in doc if token.text.lower() in final_assertive)
    pathos = sum(1 for token in doc if token.text.lower() in final_pathos)
    return pd.Series({
        "hedging_count": hedging,
        "assertive_count": assertive,
        "pathos_count": pathos,
        "word_count": len(doc)
    })

def detect_authority(doc):
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'PERSON'] and ent.text.lower() in authority_list:
            return 1
    return 0

def detect_citation(doc):
    text = doc.text.lower()
    return int(bool(re.search(r'https?://', text) or re.search(r'(according to|published in|a study by)', text)))

def detect_call_to_action(doc):
    patterns = ['you should', 'consider', 'make sure to', 'talk to your doctor', 'schedule a visit']
    return int(any(p in doc.text.lower() for p in patterns))

def detect_structured_sentences(doc):
    sents = list(doc.sents)
    if not sents:
        return 0
    complete = sum(1 for sent in sents if any(tok.dep_ == 'nsubj' for tok in sent) and any(tok.dep_ == 'ROOT' for tok in sent))
    return int(complete / len(sents) > 0.7)

def compute_ethos_logos(doc):
    ethos = detect_authority(doc) + detect_citation(doc)
    logos = detect_call_to_action(doc) + detect_structured_sentences(doc)
    return pd.Series({
        "ethos_score": ethos,
        "logos_score": logos
    })

def compute_confidence_score(row, weights=None):
    if weights is None:
        weights = {
            "assertive_rate": 0.6,
            "hedging_rate": -0.4
        }

    score = (
        weights["assertive_rate"] * row["assertive_count"] / row["word_count"]
        + weights["hedging_rate"] * row["hedging_count"] / row["word_count"]
    )
    return max(0, min(1, round(score, 3)))

def compute_persuasion_score(row, weights=None):
    if weights is None:
        weights = {
            'confidence': 0.5,  # Bucket 1
            'ethos': 0.2,       # Bucket 2
            'logos': 0.2,
            'pathos': 0.2
        }

    persuasion_score = (
        weights['confidence'] * row['confidence_score'] +
        weights['ethos'] * (row['ethos_score'] / 2) +
        weights['logos'] * (row['logos_score'] / 2) +
        weights['pathos'] * row['pathos_rate']
    )

    return max(0, min(1, round(persuasion_score, 3)))


# -- Apply all features to DataFrame --

# Run spaCy only once per row and reuse results
def process_answer(text):
    doc = nlp(text)
    liwc = count_liwc_words(doc)
    rhetorical = compute_ethos_logos(doc)
    return pd.concat([liwc, rhetorical], axis=0)

# Apply to all rows
features_df = filteredLLM["Answer"].progress_apply(process_answer)
filtered_HealthLLM = pd.concat([filteredLLM, features_df], axis=1)

# Compute final confidence score
filtered_HealthLLM["confidence_score"] = filtered_HealthLLM.apply(compute_confidence_score, axis=1)

**Going beyond simple token counts now**

In [ ]:
def extract_spacy_features(text):
    doc = nlp(text)

    # Basic linguistic features
    modals = sum(1 for token in doc if token.tag_ == 'MD')  # Modal verbs
    negations = sum(1 for token in doc if token.dep_ == "neg")
    avg_token_len = sum(len(token.text) for token in doc) / len(doc) if len(doc) > 0 else 0
    num_sentences = len(list(doc.sents))

    # Rhetorical features
    ethos_score = detect_authority(doc) + detect_citation(doc)
    logos_score = detect_call_to_action(doc) + detect_structured_sentences(doc)

    return pd.Series({
        "modals": modals,
        "negations": negations,
        "avg_token_len": avg_token_len,
        "num_sentences": num_sentences,
        "ethos_score": ethos_score,
        "logos_score": logos_score
    })

**Adding tfidf**: We added tfidf to give weightings to our findings.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_tfidf = vectorizer.fit_transform(filtered_HealthLLM['Answer'])

In [ ]:
from scipy.sparse import hstack
import numpy as np

# Step 1: Extract spaCy features into a dense array
X_spacy = filtered_HealthLLM['Answer'].apply(extract_spacy_features)
X_spacy_array = X_spacy.values  # shape: (n_samples, spaCy_feature_dim)

# Step 2: Combine sparse TF-IDF with dense spaCy features
X_combined = hstack([X_tfidf, X_spacy_array])  # shape: (n_samples, tfidf + spaCy)

# Step 3: Get rhetorical and LIWC features as a NumPy array
X_extra = filtered_HealthLLM[[
    'assertive_count',     # original raw features
    'hedging_count',
    'pathos_count',
    'ethos_score',
    'logos_score'
]].values


# Step 4: Combine all features into a single feature matrix
X_final = hstack([X_combined, X_extra])  # shape: (n_samples, tfidf + spaCy + rhetoric + LIWC)

In [ ]:
def assign_confidence_label_combined(row):
    # Confident if high score OR strong ethos/logos
    if (row['confidence_score'] >= 0.65 or
      row['ethos_score'] == 2 or row['logos_score'] == 2):
        return 'Confident'

    # Unconfident if low score OR high hedging/emotion OR no structure/credibility
    elif (row['confidence_score'] < 0.3 or
          row['pathos_count'] >= 1 or
          (row['ethos_score'] == 0 and row['logos_score'] == 0)):
        return 'Unconfident'

    # Otherwise, it's Neutral
    else:
        return 'Neutral'

filtered_HealthLLM['confidence_label'] = filtered_HealthLLM.apply(assign_confidence_label_combined, axis=1)

# Compute pathos rate
filtered_HealthLLM["pathos_rate"] = filtered_HealthLLM["pathos_count"] / filtered_HealthLLM["word_count"].clip(lower=1)
# Compute persuasion score
filtered_HealthLLM["persuasion_score"] = filtered_HealthLLM.apply(compute_persuasion_score, axis=1)

def compute_persuasion_score(row, weights=None):
    if weights is None:
        weights = {
            'confidence': 0.4,
            'ethos': 0.2,
            'logos': 0.2,
            'pathos': 0.2
        }

    persuasion_score = (
        weights['confidence'] * row['confidence_score'] +
        weights['ethos'] * (row['ethos_score'] / 2) +
        weights['logos'] * (row['logos_score'] / 2) +
        weights['pathos'] * row['pathos_rate']
    )

    return max(0, min(1, round(persuasion_score, 3)))

# Use percentiles instead of fixed numbers
low = filtered_HealthLLM["persuasion_score"].quantile(0.33)
high = filtered_HealthLLM["persuasion_score"].quantile(0.66)

def assign_persuasion_label(score):
    if score >= high:
        return 'High'
    elif score <= low:
        return 'Low'
    else:
        return 'Medium'

# Apply corrected label function
filtered_HealthLLM["persuasion_label"] = filtered_HealthLLM["persuasion_score"].apply(assign_persuasion_label)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import hstack

# Rebuild X_final with all features
X_spacy = filtered_HealthLLM['Answer'].apply(extract_spacy_features)
X_spacy_array = X_spacy.values

# Combine TF-IDF (sparse) and spaCy (dense) features
X_combined = hstack([X_tfidf, X_spacy_array])

# Include all additional numeric features including rhetorical ones
X_extra = filtered_HealthLLM[[
    'assertive_count',     # original raw features
    'hedging_count',
    'pathos_rate',
    'ethos_score',
    'logos_score'
]].values

# Final feature matrix
X_final = hstack([X_combined, X_extra])

# Labels
y = filtered_HealthLLM['persuasion_label']

# Train classifier with cross-validation
clf = RandomForestClassifier(random_state=42)
cv_scores = cross_val_score(clf, X_final, y, cv=5)

# Output scores
print(f"Cross-validation scores: {cv_scores}")
print(f"Average CV score: {cv_scores.mean():.4f}")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Features to plot
features = ['hedging_count', 'assertive_count', 'pathos_rate', 'ethos_score', 'logos_score']

# Create a subplot for each feature
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, feature in enumerate(features):
    # Bin the feature into 5 quantile-based bins
    filtered_HealthLLM[f"{feature}_bin"] = pd.qcut(filtered_HealthLLM[feature], q=5, duplicates='drop')

    # Calculate average confidence score per bin
    bin_avg_confidence = filtered_HealthLLM.groupby(f"{feature}_bin")["confidence_score"].mean().reset_index()

    # Plot as a bar chart
    sns.barplot(x=f"{feature}_bin", y="confidence_score", data=bin_avg_confidence, ax=axes[i], palette='viridis')
    axes[i].set_title(f"Confidence Score by {feature} Level")
    axes[i].set_xlabel(f"{feature} (binned)")
    axes[i].set_ylabel("Avg Confidence Score")
    axes[i].tick_params(axis='x', rotation=45)

# Hide the last subplot if unused
if len(features) < len(axes):
    axes[-1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

# NICHD Women's Health URL
BASE_URL = "https://www.nichd.nih.gov/health/topics/womenshealth"

# Headers to be polite
HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; FactCheckBot/1.0; +https://yourdomain.com/bot)"
}

# Your target topics (lowercase for matching)
target_keywords = [
    "amenorrhea", "bacterial vaginosis", "breastfeeding", "breast milk", "birth control",
    "contraceptive", "contraception", "diabetes", "endometriosis",
    "fragile x-associated primary ovarian insufficiency", "morning-after", "plan b",
    "emergency contraception", "pregnancy", "abortion", "miscarriage", "menstruation",
    "period", "fertility", "infertility", "ovulation", "iud", "pill", "uterus",
    "gynecologist", "pap smear", "hpv", "hiv", "aids", "sti", "std", "vaginal",
    "cervix", "pcos", "menopause", "estrogen", "progesterone", "labor", "delivery",
    "maternal morbidity", "maternal mortality", "pelvic floor disorder", "preeclampsia",
    "eclampsia", "vulvodynia", "uterine fibroids", "turner syndrome", "still birth"
]

def get_filtered_links(main_url, keywords):
    """Scrape subtopic links that contain any of the target keywords."""
    response = requests.get(main_url, headers=HEADERS)
    soup = BeautifulSoup(response.content, "html.parser")
    links = []

    for a in soup.select("a[href^='/health/topics/']"):
        text = a.get_text(strip=True).lower()
        href = a.get("href")
        if any(kw in text for kw in keywords):
            full_url = urljoin(main_url, href)
            links.append(full_url)

    return list(set(links))  # Deduplicate

def extract_facts_from_page(url):
    """Extract factual text (paragraphs and list items) from a subpage."""
    print(f"Scraping: {url}")
    try:
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.content, "html.parser")
        facts = []

        for tag in soup.find_all(["p", "li"]):
            text = tag.get_text(strip=True)
            if len(text.split()) >= 6 and "." in text:
                facts.append(text)

        return facts
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return []

# Step 1: Filter subtopic links
filtered_links = get_filtered_links(BASE_URL, target_keywords)

# Step 2: Scrape those filtered pages
all_facts = []
for link in filtered_links:
    facts = extract_facts_from_page(link)
    all_facts.extend(facts)
    time.sleep(1.5)  # Respectful delay

# Step 3: Preview
print(f"\n✅ Scraped {len(all_facts)} facts from {len(filtered_links)} pages.\n")
for fact in all_facts[:10]:
    print("•", fact)

# Step 4: Save (optional)
with open("nichd_filtered_facts.txt", "w") as f:
    for fact in all_facts:
        f.write(fact + "\n")


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load the model for sentence embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

with open("nichd_filtered_facts.txt", "r") as f:
    nichd_facts = [line.strip() for line in f]

# Generate embeddings for NICHD facts *outside* the loop
nichd_embeddings = model.encode(nichd_facts, convert_to_tensor=True)

# Define a function to get both confidence and accuracy labels for each response
def get_combined_label(response, persuasion_label, ethos_rate, logos_rate, nichd_embeddings, nichd_facts):
    # Semantic similarity
    resp_embedding = model.encode(response, convert_to_tensor=True)
    cosine_scores = util.cos_sim(resp_embedding, nichd_embeddings)
    max_score = cosine_scores.max().item()

    # Accuracy assessment with ethos/logos adjustments
    if max_score < 0.4:
        accuracy_label = "Possible Misinformation"
        if ethos_rate > 0.02 or logos_rate > 0.02:
            accuracy_label += " (but rhetorically persuasive)"
    elif max_score < 0.6:
        accuracy_label = "Neutral / Unclear"
    else:
        accuracy_label = "Likely aligned with NICHD"

    # Use persuasion label directly
    if persuasion_label == 'High':
        persuasion_label_desc = "Highly Persuasive"
    elif persuasion_label == 'Low':
        persuasion_label_desc = "Low Persuasion"
    else:
        persuasion_label_desc = "Moderately Persuasive"

    return f"{persuasion_label_desc} / {accuracy_label}"

# combining labels for all responses
combined_labels = []
for i, row in filtered_HealthLLM.iterrows():
    combined_label = get_combined_label(
        row['Answer'],
        row['persuasion_label'],
        row['ethos_score'],
        row['logos_score'],
        nichd_embeddings,
        nichd_facts
    )
    combined_labels.append(combined_label)

filtered_HealthLLM['combined_label'] = combined_labels


# Preview the result
print(filtered_HealthLLM[['Answer', 'combined_label']].head())


In [ ]:
# Split on the first '/' to separate 'confidence' and 'accuracy'
filtered_HealthLLM[['confidence', 'accuracy']] = filtered_HealthLLM['combined_label'].str.split(' / ', n=1, expand=True)

# If there are rows with missing values in the second part of the split, fill them with a default value (e.g., 'Unknown')
filtered_HealthLLM['confidence'].fillna('Unknown', inplace=True)
filtered_HealthLLM['accuracy'].fillna('Unknown', inplace=True)

# Inspect the data
print(filtered_HealthLLM[['combined_label', 'confidence', 'accuracy']].head())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Bar Plot of Confidence vs Accuracy
plt.figure(figsize=(10, 6))
sns.countplot(data=filtered_HealthLLM, x='accuracy', hue='confidence', palette='Set2')
plt.title('Persuasion vs Accuracy')
plt.xlabel('Accuracy')
plt.ylabel('Count of LLM Responses')
plt.xticks(rotation=30)  # Rotate x-axis labels to prevent overlap
plt.tight_layout()       # Adjusts layout to make room for rotated labels
plt.show()


In [ ]:
# Count unique combinations of 'confidence' and 'accuracy'
label_counts = filtered_HealthLLM[['confidence', 'accuracy']].value_counts()

# Display the counts
print(label_counts)
